In [108]:
import pandas as pd
import re
import requests
import shutil
import time
import geocoder
import random
from tqdm import tqdm
from pathlib import Path
from py7zr import unpack_7zarchive

In [14]:
MAIN_URL = "https://archive.org/download/stackexchange"
DATA_DIR = Path("../data")

In [11]:

try:
    shutil.register_unpack_format('7zip', ['.7z'], unpack_7zarchive)
except shutil.RegistryError:
    pass
def download_stackexchange(stack: str) -> None:
    url = f"{MAIN_URL}/{stack}.stackexchange.com.7z"
    # if file exists then skip
    if (DATA_DIR / f"{stack}.stackexchange.com.7z").exists():
        return None
    print(f"Downloading {stack} from {url}")
    data = requests.get(url)
    # check if data is valid
    with open(DATA_DIR / f"{stack}.stackexchange.com.7z", "wb") as f:
        f.write(data.content)
    
def unpack_stackexchange(stack: str) -> None:
    save_dir = DATA_DIR / stack
    if not save_dir.exists():
        save_dir.mkdir()
    if next(save_dir.iterdir(), None):
        return None
    filename = DATA_DIR / f"{stack}.stackexchange.com.7z"
    if not filename.exists():
        raise ValueError(f"File not found: {filename}")
    print(f"Unpacking {filename}")
    shutil.unpack_archive(filename, DATA_DIR / stack)

def download_and_unpack(stack: str) -> None:
    download_stackexchange(stack)
    unpack_stackexchange(stack)

In [12]:
unpack_stackexchange("politics")

Unpacking ..\data\politics.stackexchange.com.7z


In [42]:
def read_user_data(user_xml: Path, cols=["Id", "Reputation", "Location"]) -> pd.DataFrame:
    """Reads the user data from the XML file."""
    # read the xml file
    df = pd.read_xml(user_xml, "row")
    return df[cols]

userdf = read_user_data(DATA_DIR / "politics" / "Users.xml")

C:\Users\jhr\AppData\Local\Temp\ipykernel_584\1218458623.py:4: FutureWarning: In a future version of pandas all arguments of read_xml except for the argument 'path_or_buffer' will be keyword-only.
  df = pd.read_xml(user_xml, "row")


,Id,Reputation,Location
0,-1,1,on the server farm
1,1,101,"Corvallis, OR"
2,2,100,"New York, NY"
3,3,101,"New York, NY, United States"
4,4,1,None
...,...,...,...
36322,44448,1,None
36323,44449,1,None
36324,44450,131,None
36325,44451,101,"San Francisco, CA, USA"


In [43]:
userdf["meta_location"] = userdf["Location"].str.split(",").str[-1].str.strip()
userdf = userdf.assign(loc_id = userdf.groupby("meta_location").ngroup().astype("category"))

C:\Users\jhr\AppData\Local\Temp\ipykernel_584\2243356784.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  userdf["meta_location"] = userdf["Location"].str.split(",").str[-1].str.strip()


In [47]:
location_counts = userdf["meta_location"].value_counts()
common_location_filter = location_counts[location_counts > 1].index
locdf = userdf.loc[userdf["meta_location"].isin(common_location_filter), ["loc_id", "meta_location"]].drop_duplicates()
locdf


,loc_id,meta_location
0,1532.0,on the server farm
1,912.0,OR
2,833.0,NY
3,1335.0,United States
5,428.0,FL
...,...,...
32125,808.0,Multan
32891,1023.0,Quận 4
33592,1169.0,Spamalot
34705,902.0,Novosibirsk


In [49]:


session = requests.Session()

for i, row in tqdm(locdf.iterrows()):
    loc = row["meta_location"]
    if not loc:
        continue
    try:
        g = geocoder.osm(loc, session=session)
    except ValueError:
        locdf.loc[i, "country"] = None
        continue
    if g.ok:
        locdf.loc[i, "country"] = g.country
    else:
        print(f"Failed to get coordinates for {loc} with error {g.status}")
        locdf.loc[i, "country"] = None
    time.sleep(1)


0it [00:00, ?it/s]

Getting coordinates for on the server farm
Failed to get coordinates for on the server farm with error ERROR - No results found


1it [00:04,  4.80s/it]

Getting coordinates for OR


2it [00:05,  2.63s/it]

...

312it [05:43,  1.07s/it]

Getting coordinates for Italia


313it [05:45,  1.10s/it]


Getting coordinates for Antarctica


ValueError: Coords are not within the world's geographical boundary

In [38]:
simple_test = geocoder.osm("Cascadia")

In [60]:
missing_countries = locdf[locdf["country"].isna()]

for i, row in tqdm(missing_countries.iterrows()):
    loc = row["meta_location"]
    if not loc:
        continue
    try:
        g = geocoder.osm(loc, session=session)
    except ValueError:
        locdf.loc[i, "country"] = None
        continue
    if g.ok:
        locdf.loc[i, "country"] = g.country
    else:
        print(f"Failed to get coordinates for {loc} with error {g.status}")
        locdf.loc[i, "country"] = None
    time.sleep(1)

0it [00:00, ?it/s]

Failed to get coordinates for on the server farm with error ERROR - No results found


4it [00:07,  1.56s/it]

Failed to get coordinates for . with error ERROR - No results found


7it [00:09,  1.03s/it]

...

123it [02:16,  1.11s/it]

Failed to get coordinates for https://freeproversion.com/ with error ERROR - No results found


131it [02:25,  1.06s/it]

Failed to get coordinates for Spamalot with error ERROR - No results found


134it [02:28,  1.11s/it]


In [62]:
locdf.to_csv(DATA_DIR / "politics" / "location_data.csv", index=False)

In [64]:
pd.merge(userdf, locdf, on="loc_id", how="left").to_csv(DATA_DIR / "politics" / "user_loc_data.csv", index=False)

### Creating the user network
Now I'll attempt to create a user network based on the following things: 
1. Answers
2. Comments

In [65]:
postdf = pd.read_xml(DATA_DIR / "politics" / "Posts.xml", "row")

C:\Users\jhr\AppData\Local\Temp\ipykernel_584\1407501751.py:1: FutureWarning: In a future version of pandas all arguments of read_xml except for the argument 'path_or_buffer' will be keyword-only.
  postdf = pd.read_xml(DATA_DIR / "politics" / "Posts.xml", "row")


In [104]:
# filter columns to end with Id
idposts = postdf.loc[:, postdf.columns.str.endswith("Id")]

childposts = idposts.loc[idposts["ParentId"].notna(), ["Id", "ParentId", "OwnerUserId"]]
topposts = idposts[["Id", "OwnerUserId"]]

edgedf = pd.merge(childposts, topposts, left_on="ParentId", right_on="Id", how="left", suffixes=("_child", "_top"))
edgedf = edgedf[["OwnerUserId_child", "OwnerUserId_top"]].rename(columns={"OwnerUserId_child": "source", "OwnerUserId_top": "target"}).astype("Int32")
edgedf.dropna(inplace=True)
edgedf = edgedf.astype("str")


In [109]:
import networkx as nx
from pyvis.network import Network

G = nx.from_pandas_edgelist(edgedf, "source", "target", create_using=nx.DiGraph)
G = G.to_undirected()

# sample the graph to make it smaller
nodes = random.sample(list(G.nodes), 1000)
subG = G.subgraph(nodes)

# Plot the graph
net = Network("500px", "500px")
net.from_nx(subG)
net.save_graph("politics.html")


In [125]:
from networkx.algorithms import community

# Find biggest connected components
largest_cc = max(nx.connected_components(G), key=len)
subG = G.subgraph(largest_cc)


# detect communities
communities = community.louvain_communities(subG)



In [130]:
community_sizes = {i: len(c) for i, c in enumerate(communities)}
